In [13]:
import os
import requests
import json
import pandas as pd
import time

In [2]:
from web3 import Web3

In [3]:
w3 = Web3(Web3.HTTPProvider("https://fabled-purple-tree.discover.quiknode.pro/ec9f3c797cb5ee5aa38ea1ae769ca51d09cecd28/"))
w3.eth.getBlock('latest') # get latest block details
w3.eth.blockNumber # get latest block numbe

15355090

In [4]:
ETHERSCAN_KEY = '1PHP1DRI66NPF3TNPYFRF7DTS2M5Z4KZXT'

In [51]:
def eth_getblockbyno(number):

    query = 'https://api.etherscan.io/api?module=block&action=getblockreward&blockno=' + \
        str(number) + '&apikey=' + ETHERSCAN_KEY

    r = requests.post(query)
    r = json.loads(r.content)
    return r

def eth_gettxbyhash(txhash):

    query = 'https://api.etherscan.io/api?module=proxy&action=eth_getTransactionByHash&txhash=' + \
        txhash + '&apikey=' + ETHERSCAN_KEY

    r = requests.post(query)
    r = json.loads(r.content)
    return r

def eth_getntxbyadd(add, strt=0, end=99999999):
    
    query = 'https://api.etherscan.io/api?module=account&action=txlist&address=' + \
        add + '&startblock=' + str(strt) + '&endblock=' + str(end) + \
        '&offset=10000&sort=asc&apikey=' + ETHERSCAN_KEY

    r = requests.post(query)
    r = json.loads(r.content)
        
    return r

def eth_getntxbyadd_all(add, strt=0):
    res = []
    l = 10000
    while l == 10000:
        r = eth_getntxbyadd(add, strt)
        res += [r]
        l = len(r['result'])
        if l > 0: strt = r['result'][-1]['blockNumber']
        print('How many scraped on this run', l, 'next starting block', strt)
        
    return res

In [125]:
TORNADO_100E_ADDRESS = '0xA160cdAB225685dA1d56aa342Ad8841c3b53f291'
TORNADO_10E_ADDRESS = '0x910Cbd523D972eb0a6f4cAe4618aD62622b39DbF'
TORNADO_1E_ADDRESS = '0x47CE0C6eD5B0Ce3d3A51fdb1C52DC66a7c3c2936'
TORNADO_01E_ADDRESS = '0x12D66f87A04A9E220743712cE6d9bB1B5616B8Fc'
TORNADO_ROUTER_ADDRESS = '0xd90e2f925DA726b50C4Ed8D0Fb90Ad053324F31b'

TORNADO_ADDRESSES = [TORNADO_100E_ADDRESS, TORNADO_10E_ADDRESS, TORNADO_1E_ADDRESS,
                    TORNADO_01E_ADDRESS, TORNADO_ROUTER_ADDRESS]

add_to_name = {TORNADO_100E_ADDRESS:'100Eth Contract',
               TORNADO_10E_ADDRESS:'10Eth Contract',
               TORNADO_1E_ADDRESS: '1Eth Contract',
               TORNADO_01E_ADDRESS: '0.1Eth Contract',
               TORNADO_ROUTER_ADDRESS: 'Router Contract'}

In [53]:
# Lets start iding the addresses for later
add_to_id = {}
id_to_add = {}

for add in TORNADO_ADDRESSES:
    add_to_id[add] = add_to_id.get(add, len(add_to_id)+1)
    id_to_add[add_to_id[add]] = add


In [62]:
# Scrape all transactions of contracts
results = []
for add in TORNADO_ADDRESSES:
    results += [eth_getntxbyadd_all(add)]
    print('Done.')

How many scraped on this run 8385 starting block 15054146
Done.
How many scraped on this run 10000 starting block 11156054
How many scraped on this run 3859 starting block 15328182
Done.
How many scraped on this run 10000 starting block 11256710
How many scraped on this run 2421 starting block 15312828
Done.
How many scraped on this run 9686 starting block 15319496
Done.
How many scraped on this run 10000 starting block 14379487
How many scraped on this run 10000 starting block 14497523
How many scraped on this run 10000 starting block 14613152
How many scraped on this run 10000 starting block 14720895
How many scraped on this run 10000 starting block 14834886
How many scraped on this run 10000 starting block 14972650
How many scraped on this run 10000 starting block 15084844
How many scraped on this run 10000 starting block 15217300
How many scraped on this run 10000 starting block 15313773
How many scraped on this run 2191 starting block 15371036
Done.


In [73]:
# Get all results in one list
results = [r['result'] for add in results for r in add]
results = [r for add in results for r in add]

In [74]:
# Lets create a dataframe
tornado_df = pd.DataFrame(results)

# drop duplicates in case they happen in data collection process
tornado_df = tornado_df.drop_duplicates()

In [155]:
# Get a readable time field
tornado_df['time'] = tornado_df['timeStamp'].apply(lambda x: time.localtime(int(x)))

In [76]:
# Only completed transactions
tornado_df = tornado_df[tornado_df['isError'] == '0']

# We don't need the error series now
tornado_df = tornado_df.drop(['txreceipt_status', 'isError'], axis=1)

In [183]:
# Lets understand
pd.DataFrame({'counts':tmp.groupby(['to'])['functionName'].value_counts()}).unstack().fillna(0)

counts  \
functionName                               atInversebrah(int248 a, uint48[] b, uint32 c, bytes20[] d, bytes30[] e)   
to                                                                                                                   
                                                                                          4.0                        
0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc                                                0.0                        
0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936                                                0.0                        
0x910cbd523d972eb0a6f4cae4618ad62622b39dbf                                                0.0                        
0xa160cdab225685da1d56aa342ad8841c3b53f291                                                0.0                        
0xd90e2f925da726b50c4ed8d0fb90ad053324f31b                                                0.0                        

                                                                                 \
functionName                               backupNotes(bytes[] _encryptedNotes)   
to                                                                                
                                                                            0.0   
0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc                                  0.0   
0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936                                  0.0   
0x910cbd523d972eb0a6f4cae4618ad62622b39dbf                                  0.0   
0xa160cdab225685da1d56aa342ad8841c3b53f291                                  0.0   
0xd90e2f925da726b50c4ed8d0fb90ad053324f31b                                  2.0   

                                                                                \
functionName                               changeOperator(address newOperator)   
to                                                                               
                                                                           0.0   
0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc                                 1.0   
0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936                                 1.0   
0x910cbd523d972eb0a6f4cae4618ad62622b39dbf                                 1.0   
0xa160cdab225685da1d56aa342ad8841c3b53f291                                 0.0   
0xd90e2f925da726b50c4ed8d0fb90ad053324f31b                                 0.0   

                                                                                                                 \
functionName                               deposit(address _tornado, bytes32 _commitment, bytes _encryptedNote)   
to                                                                                                                
                                                                                          0.0                     
0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc                                                0.0                     
0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936                                                0.0                     
0x910cbd523d972eb0a6f4cae4618ad62622b39dbf                                                0.0                     
0xa160cdab225685da1d56aa342ad8841c3b53f291                                                0.0                     
0xd90e2f925da726b50c4ed8d0fb90ad053324f31b                                            46262.0                     

                                                                 \
functionName                               deposit(bytes32 _id)   
to                                                                
                                                            0.0   
0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc               5139.0   
0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936               6309.0   
0x910cbd523d972eb0a6f4cae4618ad62622b39dbf               7006.0   
0xa160cdab225685da1d56aa342ad8841c3b53f291           

### deposit(address _tornado, bytes32 _commitment, bytes _encryptedNote) 

##### 0x13d98d13

This function is called on the Router smart contract of Tornado.

### withdraw(address _tornado, bytes _proof, bytes32 _root, bytes32 _nullifierHash, address _recipient, address _relayer, uint256 _fee, uint256 _refund) 

##### 0x13d98d13

This function is called on the Router smart contract of Tornado.

### deposit(bytes32 _id)

This function is called by the 0.1, 1, 10, 100 Eth Tornado Contracts

### withdraw(bytes _proof, bytes32 _root, bytes32 _nullifierHash, address _recipient, address _relayer, uint256 _fee, uint256 _refund) 

This function is called by the 0.1, 1, 10, 100 Eth Tornado Contracts

